In [17]:
import pandas as pd
import numpy as np

import json
import requests

from pathlib import Path
import os
import datetime

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [18]:
# fetch FPL data online
fpl_orig_data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
fpl_data = pd.DataFrame(fpl_orig_data['elements'])
fpl_data

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,NaN,None,232223,0,0,0,0,0,4,1.5,None,0,Folarin,0.0,1,False,,None,45,232223.jpg,0.0,Balogun,1.4,False,None,a,1,3,0,0,0,0,0,0.0,0.0,Balogun,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,500,46,499,46,490,48,502,48,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,0.00,0.00,424,64,232,7,505,48,145,30,0.00,0.00
1,NaN,None,58822,0,0,0,0,0,2,1.5,None,0,Cédric,0.0,2,False,,None,40,58822.jpg,1.2,Alves Soares,0.4,False,None,a,1,3,10,0,0,0,0,0.0,2.5,Cédric,223,0,0,0,3,0,0,0,0,0,0,0,56,56.4,51.4,10.0,11.9,2,0.06,0.19,0.25,2.83,333,127,287,102,329,126,335,123,NaN,,NaN,,NaN,,0.02,0.0,0.08,0.10,1.14,1.21,517,148,9,3,349,125,285,97,0.81,0.00
2,NaN,None,153256,0,0,0,0,0,3,1.5,None,0,Mohamed,0.0,3,False,,None,45,153256.jpg,1.2,Elneny,0.2,False,None,a,1,3,6,0,0,0,0,0.0,1.3,M.Elneny,111,0,0,0,2,0,0,0,0,0,0,0,27,4.6,5.4,0.0,1.1,1,0.00,0.04,0.04,1.29,377,174,347,170,463,183,376,173,NaN,,NaN,,NaN,,0.00,0.0,0.03,0.03,1.05,1.62,350,201,158,9,351,157,399,128,0.81,0.00
3,NaN,None,438098,0,0,0,0,0,3,2.5,None,0,Fábio,0.0,4,False,,None,55,438098.jpg,1.8,Ferreira Vieira,0.1,False,None,a,1,3,40,0,0,0,0,0.0,7.3,Fábio Vieira,500,1,2,2,5,0,0,0,0,0,0,2,134,116.0,180.6,123.0,41.5,3,0.86,1.39,2.25,5.28,293,128,163,104,199,113,261,123,NaN,,NaN,,NaN,,0.15,0.0,0.25,0.40,0.95,0.90,136,89,520,206,291,125,465,153,0.54,0.36
4,NaN,None,226597,0,0,0,0,0,2,2.8,None,0,Gabriel,0.0,5,False,,None,50,226597.jpg,3.8,dos Santos Magalhães,19.9,False,None,a,1,3,146,0,0,0,0,0.0,29.2,Gabriel,3409,3,0,14,43,0,0,0,5,0,0,15,723,743.8,131.4,401.0,127.7,38,5.04,0.66,5.70,41.84,31,6,196,54,80,3,77,9,NaN,,NaN,,NaN,,0.13,0.0,0.02,0.15,1.10,1.14,250,30,420,189,67,14,18,7,1.00,0.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,NaN,None,200402,0,0,0,0,0,2,1.5,None,0,Nélson,0.0,572,False,,None,45,200402.jpg,2.1,Cabral Semedo,0.3,False,None,a,20,39,75,0,0,0,0,0.0,16.7,N.Semedo,2633,0,1,8,43,0,0,0,11,1,0,5,424,455.0,229.4,88.0,76.8,31,0.53,0.91,1.44,50.02,120,40,136,31,232,73,163,43,NaN,,NaN,,NaN,,0.02,0.0,0.03,0.05,1.71,1.47,386,96,194,121,236,90,298,103,1.06,0.27
605,NaN,None,510362,0,0,0,0,0,2,1.5,None,0,Toti António,0.0,573,False,,None,45,510362.jpg,2.5,Gomes,0.2,False,None,a,20,39,43,0,0,0,0,0.0,9.6,Toti,978,1,0,4,14,0,0,0,6,0,0,4,201,230.0,74.5,69.0,37.4,10,0.35,0.23,0.58,20.87,230,88,262,92,259,88,270,95,NaN,,NaN,,NaN,,0.03,0.0,0.02,0.05,1.92,1.29,499,144,307,169,1

In [19]:
fpl_data['element_type'].unique()

array([4, 2, 3, 1])

In [ ]:
fpl_data[~fpl_data.chance_of_playing_next_round.apply(lambda x: np.isnan(x))]

In [ ]:
np.isnan(fpl_data.loc[0,'chance_of_playing_next_round'])

In [ ]:
# for some reason, fpl team numbers are otherwise alphabetic, except Leeds comes after Leicester...
teams = ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
        'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leicester City', 'Leeds United',
         'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd',
       'Nottingham Forest', 'Southampton', 'Tottenham', 'West Ham','Wolves']

fpl_data['team'] = [teams[i-1] for i in fpl_data['team']]

In [ ]:
# change dtypes to numeric where appropriate
aux = []
for ix, col in fpl_data.items():
    aux.append(pd.to_numeric(col, errors='ignore'))
df = pd.DataFrame(aux).T

In [ ]:
df.groupby('team')['expected_goals_conceded'].sum() / 11

In [ ]:
df.groupby('team')['expected_goals'].sum()

In [ ]:
filepath = Path('../../../../season22_23/data/fpl_data/fpl_week_1.csv')
last_season_week1 = pd.read_csv(filepath)
last_season_week1.head()

In [ ]:
filepath = Path('../../../../season22_23/data/fpl_data/fpl_week_38.csv')
last_season_week38 = pd.read_csv(filepath, index_col=0)
last_season_week38.head()

In [ ]:
set(last_season_week38.columns).difference(set(last_season_week1.columns))

### 22-23 week 38 new data processing scheme

In [ ]:
# all data for season 22-23
data = []
for i in range(38):
    filepath = Path(f'../../../../season22_23/data/fpl_data/fpl_week_{i+1}.csv')
    gameweek_data = pd.read_csv(filepath, index_col=0)
    #modification_time = os.path.getmtime(filepath)
    #modification_datetime = datetime.datetime.fromtimestamp(modification_time)
    #gameweek_data['modification_datetime'] = modification_datetime
    data.append(gameweek_data)

df = pd.concat(data, ignore_index=True)
display(df.head())
display(df.tail())
display(df.shape)

In [ ]:
print('Percentage null:')
df.isnull().sum() / df.shape[0]

In [ ]:
df.name.nunique()

In [ ]:
df.id.nunique()

In [ ]:
df_grouped = df.groupby(['web_name', 'id']).count().reset_index()
df_grouped[df_grouped.id.duplicated()]

In [ ]:
df_grouped[df_grouped.id==159]

In [ ]:
df[df['id']==159]

In [ ]:
df.groupby(['web_name'])['points_per_game'].expanding(10).mean()

In [ ]:
df.groupby(['web_name'])['points_per_game'].rolling(10).mean()

In [ ]:
df['new_points'] = df.groupby(['web_name'])['total_points'].diff()
df[df.web_name=='Tierney'].head(20)[['web_name', 'total_points', 'new_points']]

# all data created last season